In [2]:
import pandas as pd
from tqdm import tqdm
import os
import numpy as np
import matplotlib.pyplot as plt
import itertools
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder

from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
# import lightgbm as lgb

from sklearn.metrics import (precision_score, recall_score, roc_auc_score, accuracy_score, mean_squared_error,
                             confusion_matrix, precision_recall_curve, roc_curve, brier_score_loss)

In [1]:
def process_bin_clas(df_train, df_test):
    df_total = pd.concat([df_train, df_test], axis=0)
    tar = df_total.iloc[:, -1]
    df_total = df_total.iloc[:, :-1]
    cat = df_total.select_dtypes(include=['object']).columns.to_list()
    df_total = pd.get_dummies(df_total, cat)
    df_total = pd.concat([df_total, tar], axis=1)
    df_train = df_total.iloc[:df_train.shape[0],:]
    df_test = df_total.iloc[df_train.shape[0]:,:]
    
    return df_train, df_test

def get_missing():
    missingValueColumns = datafrm.columns[datafrm.isnull().any()].tolist()
    percent_missing = datafrm[missingValueColumns].isnull().sum()
    print("Missing value count columnwise:")
    print(percent_missing)


data_dir ="/home/asim/Desktop/Takshshila/IOT/TestScripts/DATASETS_CLASSIFICATION/BINARY_PROBLEMS"


datasets = ["Adult", "Breast Cancer", "Credit Screening", "Ionosphere", "Liver Disorder", "Pima Indian", "Sonar"]
shorts = ["AD", "BC", "CR", "IO", "LD", "PI", "SN"]
seps = [",", "\t", "\t", "\t", "\t", "\t", ","]
header = [False, False, False, False, False, False, False]

seeds = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
for i in range(len(datasets)):
    for seed in tqdm(seeds):
        dataset = datasets[i]
        short = shorts[i]
        
        csv_path_train = os.path.join(data_dir, dataset, short + 'Train' + str(seed) + 'N.txt')
        csv_path_test = os.path.join(data_dir, dataset, short + 'Test' + str(seed) + 'N.txt')
        if header[i]:
            df_train = pd.read_csv(csv_path_train, sep = seps[i])
            df_test = pd.read_csv(csv_path_test, sep = seps[i])
        else:
            df_train = pd.read_csv(csv_path_train, sep = seps[i], header=None)
            df_test = pd.read_csv(csv_path_test, sep = seps[i], header=None)

        df_train, df_test = process_bin_clas(df_train, df_test)
        
        train_part_k = df_train.sample(frac=0.8)
        train_part_rest = df_train.drop(train_part_k.index)
        
        train_part_k.to_csv(os.path.join(data_dir, dataset, short + 'Train' + str(seed) + 'N_mod.txt'), index=False)
        train_part_rest.to_csv(os.path.join(data_dir, dataset, short + 'Val' + str(seed) + 'N_mod.txt'), index=False)
        df_test.to_csv(os.path.join(data_dir, dataset, short + 'Test' + str(seed) + 'N_mod.txt'), index=False)
    

NameError: name 'tqdm' is not defined

In [2]:
def DT_pred(X, y, details=None):
    
    # Build models with hyperparameters sets
    RSC = RandomizedSearchCV(
        estimator=DecisionTreeClassifier(),
        param_distributions={
            'criterion': ['gini', 'entropy'],
            'max_depth': range(1, 100, 10),
            'max_features': ['auto', 'sqrt', 'log2']}, 
        cv=5, n_jobs=-1, verbose = True)
    
    # Fit RandomizedSearchCV to find best hyperparameters
    search_result = RSC.fit(X, y)
#     print("Best using: ", search_result.best_params_, "Score: ", search_result.best_score_)
    if details:
        for key in search_result.best_params_:
            details[key] = search_result.best_params_[key]
        details["score"] = search_result.best_score_

    # Build models with optimized hyperparameters
    model_DT = DecisionTreeClassifier(
        criterion=search_result.best_params_["criterion"],
        max_depth=search_result.best_params_["max_depth"],
        max_features=search_result.best_params_["max_features"])

    # Split dataset into 5 consecutive folds
    kf = KFold(n_splits=5, shuffle=True, random_state=None)
    
    i = 1
    for train, test in kf.split(X):
        X_train = X.iloc[train,:]
        y_train = y.iloc[train]
        X_test = X.iloc[test]
        y_test = y.iloc[test]
        model_DT.fit(X_train, y_train)
        train_pred = model_DT.predict(X_train)
        y_pred = model_DT.predict(X_test)
        
        train_accuracy = accuracy_score(y_train, train_pred)
        train_precision = precision_score(y_train, train_pred)
        train_recall = recall_score(y_train, train_pred)
        train_auc = roc_auc_score(y_train, train_pred)
        
        test_accuracy = accuracy_score(y_test, y_pred)
        test_precision = precision_score(y_test, y_pred)
        test_recall = recall_score(y_test, y_pred)
        test_auc = roc_auc_score(y_test, y_pred)        
        
#         print('Fold '+ str(i), ':  Training accuracy: ', train_accuracy, 'Testing accuracy: ', test_accuracy)
#         print('Fold '+ str(i), ':  Training precision: ', train_precision, 'Testing precision: ', test_precision)
#         print('Fold '+ str(i), ':  Training recall: ', train_recall, 'Testing accuracy: ', test_recall)
#         print('Fold '+ str(i), ':  Training auc: ', train_auc, 'Testing auc: ', test_auc)
        
        if details:
            details['Fold '+ str(i) + "Train" + "_accuracy"] = train_accuracy
            details['Fold '+ str(i) + "Train" + "_accuracy"] = train_precision
            details['Fold '+ str(i) + "Train" + "_accuracy"] = train_recall
            details['Fold '+ str(i) + "Train" + "_accuracy"] = train_auc
            
            details['Fold '+ str(i) + "Val" + "_accuracy"] = test_accuracy
            details['Fold '+ str(i) + "Val" + "_accuracy"] = test_precision
            details['Fold '+ str(i) + "Val" + "_accuracy"] = test_recall
            details['Fold '+ str(i) + "Val" + "_accuracy"] = test_auc

        i += 1
        
    return model_DT

In [23]:
def RF_pred(X, y, details=None):
    
    # Build models with hyperparameters sets
    RSC = RandomizedSearchCV(
        estimator=RandomForestClassifier(),
        param_distributions={
            'n_estimators': range(1, 200, 10),
            'max_depth': range(1, 100, 10),
            'max_features': ['auto', 'sqrt', 'log2']}, cv=5, scoring="accuracy", n_jobs=-1)
    
    # Fit RandomizedSearchCV to find best hyperparameters
    search_result = RSC.fit(X, y.values.ravel())
#     print("Best using: ", search_result.best_params_, "Score: ", search_result.best_score_)
    if details:
        for key in search_result.best_params_:
            details[key] = search_result.best_params_[key]
        details["score"] = search_result.best_score_

    # Build models with optimized hyperparameters
    model_RF = RandomForestClassifier(
        n_estimators=search_result.best_params_["n_estimators"],
        max_depth=search_result.best_params_["max_depth"],
        max_features=search_result.best_params_["max_features"])
    

    # Split dataset into 5 consecutive folds
    kf = KFold(n_splits=5, shuffle=True, random_state=None)
    
    i = 1
    for train, test in kf.split(X):
        X_train = X.iloc[train,:]
        y_train = y.iloc[train]
        X_test = X.iloc[test]
        y_test = y.iloc[test]
        model_RF.fit(X_train, y_train.values.ravel())
        train_pred = model_RF.predict(X_train)
        y_pred = model_RF.predict(X_test)
        
        train_accuracy = accuracy_score(y_train, train_pred)
#         train_precision = precision_score(y_train, train_pred)
#         train_recall = recall_score(y_train, train_pred)
#         train_auc = roc_auc_score(y_train, train_pred)
        
        test_accuracy = accuracy_score(y_test, y_pred)
#         test_precision = precision_score(y_test, y_pred)
#         test_recall = recall_score(y_test, y_pred)
#         test_auc = roc_auc_score(y_test, y_pred)
        
#         print('Fold '+ str(i), ':  Training accuracy: ', train_accuracy, 'Testing accuracy: ', test_accuracy)
#         print('Fold '+ str(i), ':  Training precision: ', train_precision, 'Testing precision: ', test_precision)
#         print('Fold '+ str(i), ':  Training recall: ', train_recall, 'Testing accuracy: ', test_recall)
#         print('Fold '+ str(i), ':  Training auc: ', train_auc, 'Testing auc: ', test_auc)
        
        if details:
            details['Fold '+ str(i) + "Train" + "_accuracy"] = train_accuracy
#             details['Fold '+ str(i) + "Train" + "_precision"] = train_precision
#             details['Fold '+ str(i) + "Train" + "_recall"] = train_recall
#             details['Fold '+ str(i) + "Train" + "_auc"] = train_auc
            
            details['Fold '+ str(i) + "Val" + "_accuracy"] = test_accuracy
#             details['Fold '+ str(i) + "Val" + "_precision"] = test_precision
#             details['Fold '+ str(i) + "Val" + "_recall"] = test_recall
#             details['Fold '+ str(i) + "Val" + "_auc"] = test_auc
        i += 1

    return model_RF

In [4]:
def plot_performance(y_test, y_pred, y_pred_prob):


    test_fpr, test_tpr, _ = roc_curve(y_test, y_pred_prob)
    precision, recall, _ = precision_recall_curve(y_test, y_pred)
    
    # ROC Curve
    fig = plt.figure(1, figsize=(10,5))
    plt.subplot(1, 2, 1)
    plt.plot(test_fpr, test_tpr, label="ROC (area = %0.4f)" % roc_auc_score(y_test, y_pred), color="blue", lw=2)
    plt.plot([0, 1], [0, 1], "k--")
    plt.legend(loc="lower right")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve")
    
    
    # Precision Recall Curve
    plt.subplot(1, 2, 2)
    plt.plot(recall, precision, marker='.', color="blue", lw=2)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title("Precision Recall Curve")
    
    
    plt.tight_layout()
    plt.show()

In [5]:
def plot_confusion(y_test, y_pred):   
    
    cm = confusion_matrix(y_test, y_pred)
    
    # Confusion Matrix
    fig = plt.figure(figsize=(10,10))
    ax =  fig.add_subplot(1,1,1, adjustable='box', aspect=1)
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.colorbar()

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')

    np.set_printoptions(precision=2)
    plt.show()

In [ ]:
data_dir =\
    r"C:\Users\arpit\workspace\setup-stuff\gateway_and_dataset\Classification\DATASETS_CLASSIFICATION\BINARY_PROBLEMS"



datasets = ["Adult", "Breast Cancer", "Credit Screening", "Ionosphere", "Liver Disorder", "Pima Indian", "Sonar"]
shorts = ["AD", "BC", "CR", "IO", "LD", "PI", "SN"]
seeds = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
for i in range(len(datasets)):
    df = pd.DataFrame()
    for seed in tqdm(seeds):
        dataset = datasets[i]
        short = shorts[i]
#         print(dataset, short)
        csv_path_train = os.path.join(data_dir, dataset, short + 'Train' + str(seed) + 'N_mod.txt')
        csv_path_val = os.path.join(data_dir, dataset, short + 'Val' + str(seed) + 'N_mod.txt')
        csv_path_test = os.path.join(data_dir, dataset, short + 'Test' + str(seed) + 'N_mod.txt')
        df_train = pd.read_csv(csv_path_train)
        df_val = pd.read_csv(csv_path_val)
        df_train = pd.concat([df_train, df_val], axis=0)
        df_test = pd.read_csv(csv_path_test)
        X = df_train.iloc[:, :-1]
        y = df_train.iloc[:, -1]
        X_test = df_test.iloc[:, :-1]
        y_test = df_test.iloc[:, -1]
        label_encoder = LabelEncoder()
        label_encoder.fit(pd.concat([y, y_test], axis=0))
        y = pd.DataFrame(label_encoder.transform(y))
        y_test = pd.DataFrame(label_encoder.transform(y_test))

        details = {'dataset': dataset, 'seed': seed}

#         model_DT = DT_pred(X, y.ravel())

        model_RF = RF_pred(X, y, details)
        importances = model_RF.feature_importances_
        indices = np.argsort(importances)[::-1]
        top_k = 10
        top_indices = indices[:top_k]
        details['best_feature_list'] = np.array(X.columns)[indices][0:top_k]

#         print("Feature_importance: \n")
#         print(np.array(X.columns)[indices][0:top_k])
#         print(importances[top_indices])
        # Random Forest performance
        y_pred = model_RF.predict(X_test)
        y_pred_prob = model_RF.predict_proba(X_test)
        y_pred_prob = y_pred_prob[:, 1]

#         plot_performance(y_test, y_pred, y_pred_prob)
#         plot_confusion(y_test, y_pred)
        
        test_accuracy = accuracy_score(y_test, y_pred)
#         test_precision = precision_score(y_test, y_pred)
#         test_recall = recall_score(y_test, y_pred)
#         test_auc = roc_auc_score(y_test, y_pred)
        details["Test_accuracy"] = test_accuracy
#         details["Test_precision"] = test_precision
#         details["Test_recall"] = test_recall
#         details["Test_auc"] = test_auc

        df = df.append(details, ignore_index=True)
        filepath = Path(dataset + '_RF.csv')
        filepath.parent.mkdir(parents=True, exist_ok=True)
        df.to_csv(filepath, index=False)

In [4]:
def process_bin_clas(df_train, df_test):
    df_total = pd.concat([df_train, df_test], axis=0)
    tar = df_total.iloc[:, -1]
    df_total = df_total.iloc[:, :-1]
    cat = df_total.select_dtypes(include=['object']).columns.to_list()
    df_total = pd.get_dummies(df_total, cat)
    df_total = pd.concat([df_total, tar], axis=1)
    df_train = df_total.iloc[:df_train.shape[0],:]
    df_test = df_total.iloc[df_train.shape[0]:,:]
    return df_train, df_test

def get_missing():
    missingValueColumns = datafrm.columns[datafrm.isnull().any()].tolist()
    percent_missing = datafrm[missingValueColumns].isnull().sum()
    print("Missing value count columnwise:")
    print(percent_missing)


data_dir ="/home/asim/Desktop/Takshshila/IOT/TestScripts/DATASETS_CLASSIFICATION"



datasets = ['data11_cns_splits','data15_occupanydetection_splits','data19_musk_splits','data22_connect_splits',
            'data26_sales_splits','data2_letter_splits','data6_shuttle_splits','data12_srbct_splits',
            'data16_poker_splits','data1_credit_splits','data23_forestcover_splits','data27_yeast_splits',
            'data7_penbased_splits','data13_lymphoma_splits','data17_skinseg_splits', 'data20_carvana_splits',
            'data24_voice_splits','data28_theorem_splits','data4_activity_splits', 'data8_income_splits',
            'data10_brain_splits','data14_prostrate_splits','data18_challenges_splits', 'data21_gimmecredit_splits',
            'data29_dota_splits','data5_bank_splits','data9_amlall_splits'] # 'data25_arrythmia_splits',

seeds = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
for i in range(len(datasets)):
    print(datasets[i])
    for seed in tqdm(seeds):
        dataset = datasets[i]
        
        remove_split = "_".join(dataset.split("_")[:-1])
        csv_path_train = os.path.join(data_dir, dataset, remove_split + '_train' + str(seed) + '.csv')
        csv_path_test = os.path.join(data_dir, dataset, remove_split + '_test' + str(seed) + '.csv')
        
        df_train = pd.read_csv(csv_path_train)
        df_test = pd.read_csv(csv_path_test)
        
        df_train, df_test = process_bin_clas(df_train, df_test)
        x, y = df_train.iloc[: , :-1], df_train.iloc[: , -1]
        if dataset == "data18_challenges_splits":
            x, y = df_train.iloc[: , 1:], df_train.iloc[: , 0]
            df_train = pd.concat([x, y], axis=1)
#         x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=4, stratify=y)
#         train_part_k = pd.concat([x_train, y_train], axis=1)
#         train_part_rest = pd.concat([x_test, y_test], axis=1)
        
        df_train.to_csv(os.path.join(data_dir, dataset, remove_split + '_train' + str(seed) + '_rf.csv'), index=False)
        df_test.to_csv(os.path.join(data_dir, dataset, remove_split + '_test' + str(seed) + '_mod.csv'), index=False)
#         train_part_rest.to_csv(os.path.join(data_dir, dataset, remove_split + '_test' + str(seed) + '_mod.csv'), index=False)

data11_cns_splits


100%|███████████████████████████████████████████| 10/10 [00:09<00:00,  1.04it/s]


data15_occupanydetection_splits


100%|███████████████████████████████████████████| 10/10 [00:00<00:00, 13.62it/s]


data19_musk_splits


...

100%|███████████████████████████████████████████| 10/10 [00:15<00:00,  1.52s/it]


data5_bank_splits


100%|███████████████████████████████████████████| 10/10 [00:01<00:00,  7.59it/s]


data9_amlall_splits


100%|███████████████████████████████████████████| 10/10 [00:09<00:00,  1.00it/s]


In [25]:
data_dir =\
    r"C:\Users\arpit\workspace\setup-stuff\gateway_and_dataset\Classification\DATASETS_CLASSIFICATION"
datasets = [# 'data19_musk_splits','data15_occupanydetection_splits',
            'data11_cns_splits', 'data22_connect_splits', 'data26_sales_splits',
            'data2_letter_splits','data6_shuttle_splits', # 'data12_srbct_splits',
            'data16_poker_splits','data1_credit_splits','data23_forestcover_splits','data27_yeast_splits',
            'data7_penbased_splits','data13_lymphoma_splits', 'data17_skinseg_splits', 'data20_carvana_splits',
            'data24_voice_splits','data28_theorem_splits','data4_activity_splits', 'data8_income_splits',
            'data18_challenges_splits', 'data21_gimmecredit_splits', 'data29_dota_splits','data5_bank_splits']
seeds = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

for i in range(len(datasets)):
    df = pd.DataFrame()
    for seed in tqdm(seeds):
        dataset = datasets[i]
        remove_split = "_".join(dataset.split("_")[:-1])
        
        csv_path_train = os.path.join(data_dir, dataset, remove_split + '_train' + str(seed) + '_mod.csv')
        csv_path_test = os.path.join(data_dir, dataset, remove_split + '_test' + str(seed) + '_mod.csv')
        
        df_train = pd.read_csv(csv_path_train)
        df_test = pd.read_csv(csv_path_test)

        X = df_train.iloc[:, :-1]
        y = df_train.iloc[:, -1]
        X_test = df_test.iloc[:, :-1]
        y_test = df_test.iloc[:, -1]
        label_encoder = LabelEncoder()
        label_encoder.fit(pd.concat([y, y_test], axis=0))
        y = pd.DataFrame(label_encoder.transform(y)).apply(pd.to_numeric)
        y_test = pd.DataFrame(label_encoder.transform(y_test)).apply(pd.to_numeric)

        details = {'dataset': dataset, 'seed': seed}

#         model_DT = DT_pred(X, y)

        model_RF = RF_pred(X, y, details)
        importances = model_RF.feature_importances_
        indices = np.argsort(importances)[::-1]
        top_k = 10
        top_indices = indices[:top_k]
        details['best_feature_list'] = np.array(X.columns)[indices][0:top_k]

#             print("Feature_importance: \n")
#             print(np.array(X.columns)[indices][0:top_k])
#             print(importances[top_indices])
        # Random Forest performance
        y_pred = model_RF.predict(X_test)
        y_pred_prob = model_RF.predict_proba(X_test)
        y_pred_prob = y_pred_prob[:, 1]

#         plot_performance(y_test, y_pred, y_pred_prob)
#         plot_confusion(y_test, y_pred)

        test_accuracy = accuracy_score(y_test, y_pred)
#         test_precision = precision_score(y_test, y_pred)
#         test_recall = recall_score(y_test, y_pred)
#         test_auc = roc_auc_score(y_test, y_pred)
        details["Test_accuracy"] = test_accuracy
#         details["Test_precision"] = test_precision
#         details["Test_recall"] = test_recall
#         details["Test_auc"] = test_auc

        df = df.append(details, ignore_index=True)
        filepath = Path(dataset + '_RF_res.csv')
        filepath.parent.mkdir(parents=True, exist_ok=True)
        df.to_csv(filepath, index=False)


  0%|                                                                                           | 0/10 [00:00<?, ?it/s]C:\Users\arpit\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
 10%|████████▎                                                                          | 1/10 [00:10<01:33, 10.37s/it]C:\Users\arpit\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
 20%|████████████████▌                                                                  | 2/10 [00:15<00:57,  7.25s/it]C:\Users\arpit\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
 30%|████████████████████████▉                               